**Note: This notebook must be changed by the student. As it is now, it does not approximate the solution to the Poisson problem!**

# 2D Solver for the time-harmonic Maxwell equation on the Unit Square
## ... using Psydac's de Rham interface

In this exercise we write a solver for the 2D time-harmonic Maxwell equation on the unit square using Psydac's de Rham interface.

\begin{align*}
    \boldsymbol{curl}curl\boldsymbol{E} - \omega^2\boldsymbol{E} &= \boldsymbol{F} \quad \text{ in }\Omega=]0,1[^2, \\
    \boldsymbol{n}\times\boldsymbol{E} &= 0 \quad \text{ on }\partial\Omega,
\end{align*}
for the specific choice
\begin{align*}
    \omega &= 1.5 \\
    \alpha &= -\omega^2 \\
    \boldsymbol{F}(x, y) &= \left(\begin{matrix}
    (\alpha+\pi^2)\sin(\pi y) - \pi^2\cos(\pi x)\sin(\pi y) \\
    (\alpha+\pi^2)\sin(\pi x)\cos(\pi y)
    \end{matrix}\right)
\end{align*}

## The Variational Formulation

The corresponding variational formulation reads

\begin{align*}
    \text{Find }\boldsymbol{E}\in V\coloneqq H_0(curl;\Omega)\text{ such that } \\
    a(\boldsymbol{E}, \boldsymbol{v}) = l(\boldsymbol{v})\quad\forall\ \boldsymbol{v}\in V
\end{align*}

where 
- $a(\boldsymbol{E},\boldsymbol{v}) \coloneqq \int_{\Omega} (\nabla\times\boldsymbol{E})(\nabla\times\boldsymbol{v}) + \alpha \boldsymbol{E}\cdot\boldsymbol{v} ~ d\Omega$ ,
- $l(\boldsymbol{v}) := \int_{\Omega} \boldsymbol{F}\cdot\boldsymbol{v} ~ d\Omega$.

## Discrete Model using de Rham objects

In [ ]:
from sympde.calculus      import dot
from sympde.topology      import elements_of, Square, Derham
from sympde.expr.expr     import LinearForm, BilinearForm, integral

domain  = Square('S', bounds1=(0, 1), bounds2=(0, 1))
derham  = Derham(domain, sequence=['h1', 'hcurl', 'l2'])

V1      = derham.V1
V2      = derham.V2

u1, v1  = elements_of(V1, names='u1, v1')
u2, v2  = elements_of(V2, names='u2, v2')

# bilinear forms corresponding to V1 and V2 mass matrices
m1      = BilinearForm((u1, v1), integral(domain,      dot(u1, v1)))
m2      = BilinearForm((u2, v2), integral(domain,      u2 * v2))

# linear form corresponding to the rhs
from sympy  import pi, sin, cos, Tuple, Matrix
omega   = 1.5
alpha   = -omega**2
x,y     = domain.coordinates
F       = Tuple( (alpha + pi**2) * sin(pi*y) - pi**2 * sin(pi*y) * cos(pi*x),
                 (alpha + pi**2) * sin(pi*x) * cos(pi*y) )

expr    = dot(F, v1)
l       = LinearForm(v1, integral(domain, expr))

In [ ]:
from psydac.api.discretization  import discretize
from psydac.api.settings        import PSYDAC_BACKEND_GPYCCEL

backend = PSYDAC_BACKEND_GPYCCEL

In [ ]:
ncells  = [16, 16]  # Bspline cells
degree  = [3, 3]    # Bspline degree

In [ ]:
domain_h    = discretize(domain, ncells=ncells, periodic=[False, False])
derham_h    = discretize(derham, domain_h, degree=degree)

V1_h        = derham_h.V1
V2_h        = derham_h.V2

# Exterior Derivative operators (grad and curl)
G, C        = derham_h.derivatives_as_matrices

# Mass matrices
m1_h        = discretize(m1,    domain_h, (V1_h, V1_h), backend=backend)
m2_h        = discretize(m2,    domain_h, (V2_h, V2_h), backend=backend)

M1          = m1_h.assemble()
M2          = m2_h.assemble()

# System Matrix A
A           = M1

# rhs vector f
l_h         = discretize(l,     domain_h, V1_h,         backend=backend)
f           = l_h.assemble()

## Boundary Conditions

We choose to apply a projection method. For that matter, we construct the projection matrix $\mathbb{P}_0$ and its counterpart $\mathbb{P}_{\Gamma} = \mathbb{I} - \mathbb{P}_0$.

In [ ]:
from utils import HcurlBoundaryProjector2D
from psydac.linalg.basic import IdentityOperator

P_Hcurl     = HcurlBoundaryProjector2D(V1, V1_h.coeff_space)

I1          = IdentityOperator(V1_h.coeff_space)
P_Gamma     = I1 - P_Hcurl

In [ ]:
A_bc        = P_Hcurl @ A @ P_Hcurl + P_Gamma
f_bc        = P_Hcurl @ f

## Solving the PDE

In [ ]:
import  time

from    psydac.linalg.solvers import inverse

tol     = 1e-9
maxiter = 1000

A_bc_inv  = inverse(A_bc, 'cg', tol=tol, maxiter=maxiter)

t0      = time.time()
E_h     = A_bc_inv @ f_bc
t1      = time.time()

## Computing the error norm

As the analytical solution is available, we want to compute the $L^2$ norm of the error.
In this example, the analytical solution is given by

$$
\boldsymbol{E}_{ex}(x, y) = \left(\begin{matrix} 
                \sin(\pi y) \\
                \sin(\pi x)\cos(\pi y)
                \end{matrix}\right)
$$

In [ ]:
from psydac.fem.basic   import FemField

from sympde.expr        import Norm

E_ex            = Tuple(sin(pi*y), sin(pi*x)*cos(pi*y))
E_h_FemField    = FemField(V1_h, E_h)

error           = Matrix([u1[0] - E_ex[0], u1[1] - E_ex[1]])

# create the formal Norm object
l2norm          = Norm(error, domain, kind='l2')

# discretize the norm
l2norm_h        = discretize(l2norm, domain_h, V1_h, backend=backend)

# assemble the norm
l2_error        = l2norm_h.assemble(u1=E_h_FemField)

# print the result
print( '> Grid          :: [{ne1},{ne2}]'.format( ne1=ncells[0], ne2=ncells[1]) )
print( '> Degree        :: [{p1},{p2}]'  .format( p1=degree[0], p2=degree[1] ) )
print( '> CG info       :: ',A_bc_inv.get_info() )
print( '> L2 error      :: {:.2e}'.format( l2_error ) )
print( '' )
print( '> Solution time :: {:.3g}'.format( t1-t0 ) )

## Visualization

We plot the true solution $\boldsymbol{E}_{ex}$, the approximate solution $\boldsymbol{E}_h$ and the error function $|\boldsymbol{E}_{ex} - \boldsymbol{E}_h|$.

In [ ]:
from sympy import lambdify

from utils import plot

E_ex_x   = lambdify((x, y), E_ex[0])
E_ex_y   = lambdify((x, y), E_ex[1])
E_h_x    = E_h_FemField[0]
E_h_y    = E_h_FemField[1]
error_x  = lambda x, y: abs(E_ex_x(x, y) - E_h_x(x, y))
error_y  = lambda x, y: abs(E_ex_y(x, y) - E_h_y(x, y))

plot(gridsize_x     = 100, 
     gridsize_y     = 100, 
     title          = r'Approximation of Solution $\boldsymbol{E}$, first component', 
     funs           = [E_ex_x, E_h_x, error_x], 
     titles         = [r'$(\boldsymbol{E}_{ex})_1(x,y)$', r'$(\boldsymbol{E}_h)_1(x,y)$', r'$|(\boldsymbol{E}_{ex}-\boldsymbol{E}_h)_1(x,y)|$'],
     surface_plot   = True
)

plot(gridsize_x     = 100,
     gridsize_y     = 100,
     title          = r'Approximation of Solution $\boldsymbol{E}$, second component',
     funs           = [E_ex_y, E_h_y, error_y],
     titles         = [r'$(\boldsymbol{E}_{ex})_2(x,y)$', r'$(\boldsymbol{E}_h)_2(x,y)$', r'$|(\boldsymbol{E}_{ex}-\boldsymbol{E}_h)_2(x,y)|$'],
     surface_plot   = True
)